In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import os
from qiskit import QuantumCircuit

qasm_dir = "/Users/debarthipal/Library/CloudStorage/OneDrive-IBM/Desktop/VS files/cutting+obp_new_git_env/QASM_circuits_small"
qasm_files = sorted([
    f for f in os.listdir(qasm_dir)
    if f.endswith(".qasm")
])
print(qasm_files)

['adder_n10.qasm', 'adder_n4.qasm', 'basis_change_n3.qasm', 'basis_trotter_n4.qasm', 'bell_n4.qasm', 'cat_state_n4.qasm', 'deutsch_n2.qasm', 'dnn_n2.qasm', 'dnn_n8.qasm', 'error_correctiond3_n5.qasm', 'fredkin_n3.qasm', 'grover_n2.qasm', 'hhl_n7.qasm', 'hs4_n4.qasm', 'iswap_n2.qasm', 'linearsolver_n3.qasm', 'qaoa_n3.qasm', 'qaoa_n6.qasm', 'qec_en_n5.qasm', 'qft_n4.qasm', 'qpe_n9.qasm', 'qrng_n4.qasm', 'quantumwalks_n2.qasm', 'sat_n7.qasm', 'simon_n6.qasm', 'teleportation_n3.qasm', 'toffoli_n3.qasm', 'variational_n4.qasm', 'vqe_n4.qasm', 'wstate_n3.qasm']


In [19]:
from circuit_cutting import perform_cutting
from qiskit_ibm_runtime.fake_provider import FakeTorino
backend = FakeTorino()


In [20]:
from qiskit_addon_obp.utils.simplify import OperatorBudget
op_budget = OperatorBudget(max_qwc_groups=5)


In [21]:
import json
from qiskit.quantum_info import SparsePauliOp
circ_dic={}
for index, file in enumerate(qasm_files, start=1):

    n=QuantumCircuit.from_qasm_file(file).num_qubits
    observable_terms = [
            "I"*(i-1) + "Z" + "I"*(n - i)
            for i in range(1, n+1)
        ]
    observable = SparsePauliOp(observable_terms, coeffs=[1/(n)] * (n))
    circuit=QuantumCircuit.from_qasm_file(file)

    number_subexp_without_obp, number_subexp_with_obp = perform_cutting(circuit, observable,backend,op_budget)
    circ_dic[index] = {"filename":file, 
                          "num_subexp_without_obp": number_subexp_without_obp, 
                          "num_subexp_with_obp":number_subexp_with_obp,
                          }
    
    json_obj=json.dumps(circ_dic, indent=4)
    with open("cuts_with_and_without_obp_for_max_qwc_group_5.json", "w") as f:
        f.write(json_obj)   #Saving to a JSON file

128
<class 'NoneType'>
432
<class 'NoneType'>
2592
<class 'NoneType'>
<class 'NoneType'>
12
<class 'NoneType'>
12
<class 'NoneType'>
1
<class 'NoneType'>
1
<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>
4
4
<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>
None
<class 'NoneType'>
2592
<class 'NoneType'>
1
<class 'NoneType'>
<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>
None
2
<class 'NoneType'>
1
<class 'NoneType'>
72
<class 'NoneType'>
576
<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>
252
252
<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>
None
576
<class 'NoneType'>
2304
<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>
4
4
<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>
None
4
<class 'NoneType'>
1
<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>
4
4
<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>
None
6
<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>
24
24
12
<class 'NoneType'>
576
<class 'NoneType'>
72
<class 'No

In [6]:
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# import numpy as np
# from matplotlib.colors import ListedColormap
# from matplotlib.patches import Patch

# # Custom color map: 0=orange, 1=green, 2=purple, 3=skyblue
# colors = ListedColormap(['orange', 'green', 'purple', 'white'])

# # Convert dictionary to DataFrame and fill Nones with 0
# print
# df = pd.DataFrame(circ_dic).T.fillna(0)

# # Extract values
# before = df["depth_transpile_before_obp"].to_numpy(dtype=float)
# after = df["depth_transpile_after_bp"].to_numpy(dtype=float)
# labels = df["filename"].tolist()

# # Init matrix with 3 (default color: white)
# matrix = np.full((len(labels), len(labels)), 3)

# # Apply logic only to diagonal where filenames match
# for i in range(len(labels)):
#     if labels[i] == labels[i]:  # always true, but clarifies intent
#         if before[i] == after[i]:
#             matrix[i, i] = 0  # orange
#         elif after[i] > before[i]:
#             matrix[i, i] = 1  # green
#         elif after[i] < before[i] and ((before[i] is None )!=(after[i] is None)):
#             matrix[i, i] = 2  # purple

# # Plot heatmap
# plt.figure(figsize=(12, 10))
# sns.heatmap(matrix, cmap=colors, cbar=False,linewidths=0.5,xticklabels=labels, yticklabels=labels)
# plt.title("Benchpress small circuits (IBM Torino <ZZ>)")

# # Legend
# legend_elements = [
#     Patch(facecolor='orange', label='Equal (Before = After)'),
#     Patch(facecolor='green', label='After > Before'),
#     Patch(facecolor='purple', label='After < Before')
# ]
# plt.legend(handles=legend_elements)
# plt.tight_layout()
# plt.savefig('small_circuits.png')
# plt.show()
